In [1]:
def input(path):
    with open(path) as f:
        firstline = f.readline().split()
        N = int(firstline[0])
        D = int(firstline[1])
        alpha = int(firstline[2])
        beta = int(firstline[3])

        person_day_off = []
        for i in range(N):
            line = f.readline()
            person_day_off.append([int(day_off) for day_off in line.split()[:-1]])

        return N, D, alpha, beta, person_day_off

In [31]:
path = 'miniproject-18\data.txt'
N, D, alpha, beta, person_day_off = input(path)

print(N, D, alpha, beta)
print(person_day_off)

9 5 1 3
[[], [4], [], [], [], [], [], [5], []]


# MIPS

In [32]:
from ortools.linear_solver import pywraplp

In [33]:
solver = pywraplp.Solver.CreateSolver('CBC')
INF = solver.infinity()
big_num = 1e6

In [34]:
x = [[[solver.IntVar(0, 1, 'x[' + str(person) + '][' + str(day) + '][' + str(shift) + ']') for shift in range(5)] for day in range(D)] for person in range(N)]

y = solver.IntVar(0, D, 'y')

In [41]:
for day in range(D):
    for person in range(N):
        c = solver.Constraint(1, 1)
        for shift in range(5):
            c.SetCoefficient(x[person][day][shift], 1)

for day in range(D):
    for shift in range(5):
        c = solver.Constraint(alpha, beta)
        for person in range(N):
            c.SetCoefficient(x[person][day][shift], 1)

for person in range(N):
        c = solver.Constraint(-INF, 0)
        for day in range(D):
            c.SetCoefficient(x[person][day][3], 1)
            c.SetCoefficient(y, -1)

for day in range(D - 1):
    for person in range(N):
            c1 = solver.Constraint(-INF, big_num - 1)
            c2 = solver.Constraint(-INF, big_num + 1)
            c1.SetCoefficient(x[person][day][3], big_num)
            c1.SetCoefficient(x[person][day + 1][4], -1)
            c2.SetCoefficient(x[person][day][3], big_num)
            c2.SetCoefficient(x[person][day + 1][4], 1)

for person, list_day_off in enumerate(person_day_off):
    for day in range(D):
        if day + 1 in list_day_off:
            solver.add(x[person][day][4] == 1)
        else:
            solver.add(x[person][day][4] == 0)

In [42]:
obj = solver.Objective()
obj.SetCoefficient(y, -1)
obj.SetMaximization()

In [43]:
result_status = solver.Solve()

# The problem has an optimal solution.

assert result_status == pywraplp.Solver.OPTIMAL
print(- solver.Objective().Value())

for person in range(N):
    for day in range(D):
        for shift in range(5):
            if x[person][day][shift].solution_value() == 1:
                print(shift if shift != 4 else '_', end = ' ')
    print()

2.0
1 2 1 2 0 
3 _ 3 _ 2 
0 2 1 0 3 
0 1 0 2 2 
0 1 1 0 1 
2 0 2 0 1 
1 2 0 2 2 
_ 3 _ 3 _ 
1 0 2 1 1 


# CP

In [1]:
from ortools.sat.python import cp_model

In [2]:
model = cp_model.CpModel()

In [8]:
x = [[[model.NewIntVar(0, 1, 'x[' + str(person) + '][' + str(day) + '][' + str(shift) + ']') for shift in range(5)] for day in range(D)] for person in range(N)]

y = model.NewIntVar(0, var, 'y')

NameError: name 'var_upper_bound' is not defined

In [ ]:
for 